# europäische Grüne Deal

I'm going to fetch the official green deal communication in german:


In [1]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import unicodedata
import re

In [2]:
URL = "https://eur-lex.europa.eu/legal-content/DE/TXT/HTML/?uri=CELEX:52019DC0640&from=EN"
law = "Europäisches Klimagesetz"
r = requests.get(URL) 
soup = BeautifulSoup(r.content, 'html5lib') 

In the Eu Communication, starting at "Abbildung1: der Europäische Grüne Deal", all the paragraphs are one different topic, so each one of these paragraphs will be one different document.
I'm consiously removing the "p.text2" tags, which are quotes and getting P.normal and P.Question

In [3]:
table = soup.findAll('p', attrs = {'class':'Normal'})
table2 = soup.findAll('p', attrs = {'class':'Question'})

#juntando os paragrafos que estavam como normal e os que estavam como question:

for i in table2:
    table.append(i)

df_com = pd.DataFrame(columns = ['original_id','gesetzt', 'artikel', 'absatz', 'text'])

for element in table:
    text = element.get_text()
    text = unicodedata.normalize('NFKD', text)   # removind \xa0
    re.sub("\n", " ", text)
    re.sub("\s{1,}", " ", text)
    df_temp = pd.DataFrame({'original_id' : ["NA"],
                            'gesetzt': ["eu_gruene_deal_com"],
                            'artikel': ["NA"], 
                            'absatz' : ["NA"],
                            'text': [text]})
    df_com = pd.concat([df_com, df_temp])

#Vou primeiro criar um index na mao para esse db:

index = pd.Index(range(0,len(df_com)))
df_com = df_com.set_index(index)

tentei tirar os metacharacters de tudo quanto é jeito mas nao deu. O jeito é move on e esperar que isso nao interfira no meu TM

In [4]:
retirar = list(range(95,89, -1)) + list(range(8,-1, -1))   # eu fiz o loop ao contrário só por precaucao, mas os indices nao se modificam quando um item é dropado.

for i in retirar:      # reverse loop
    df_com.drop(i, axis=0, inplace=True)

In [5]:
#Salvando o banco com esse artigo:

df_com.to_csv("eu_gruene_deal_com.csv")